In [2]:
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np

'''
# 함수 설정
def reorderPts(pts):
    idx = np.lexsort((pts[:, 1], pts[:, 0]))  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
    pts = pts[idx]  # x좌표로 정렬

    if pts[0, 1] > pts[1, 1]:
        pts[[0, 1]] = pts[[1, 0]]

    if pts[2, 1] < pts[3, 1]:
        pts[[2, 3]] = pts[[3, 2]]

    return pts
'''

# 이미지 불러오기
src = cv2.imread('./Data/Example Student ID Card2.jpg')

if src is None:
    print('image load failed')
    sys.exit()
    
# 이미지 resize 0.5 * 0.5
src = cv2.resize(src, (0,0), fx = 0.5, fy = 0.5)

print(src.shape)

# 이미지 사본 만들기
copy = src.copy()

# 흑백으로 변환
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

'''
# 이진화
ret, src_binary= cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
'''

# 이진화 cv2.adaptivethreshold
threshold1 = cv2.adaptiveThreshold(src_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# 외곽선 찾기
contours, _ = cv2.findContours(threshold1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print(len(contours))

# 외곽선 근사화 contours가 list이기 때문에 for 사용
for pts in contours: # pts = numpy.ndarray (K * 1 * 2 로 되어있음)
    if cv2.contourArea(pts) < 1000:
        continue
        
    # 외곽선 근사화(단순화)
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
    
    # 컨벡스가 아니면 제외
    if not cv2.isContourConvex(approx) or len(approx) != 4:
        continue
    
    w, h = 900, 500
    # 네 꼭지점이 있는 사각형의 모양 변경 (찌그러뜨리기 or 팽팽하게 펴기)
    src_resized = np.array([[approx[0, 0, :]], [approx[1, 0, :]], 
                            [approx[2, 0, :]], [approx[3, 0, :]]],dtype=np.float32)
    
    # 우측 상단, 좌측 상단, 우측 하단, 좌측 하단 순서로 입력
    dst_resized = np.array([[0, h], [w, h], [w, 0], [0, 0]]).astype(np.float32)
    
    # getPerspectiveTransform(4개의 원본 좌표, 4개의 결과 좌표)
    pers = cv2.getPerspectiveTransform(src_resized, dst_resized) 
    
    #  warpPerspective(입력,  getPerspective의 값, 출력이미지의 크기)
    dst = cv2.warpPerspective(src, pers, (w , h), flags=cv2.INTER_CUBIC)

    cv2.polylines(copy, pts, True, (0, 0, 255), 2)  # 닫힌 외곽선이면 True, 컬러는 (b, g, r) 

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('src_binary', threshold1)
cv2.imshow('contours', copy)
cv2.waitKey()
cv2.destroyAllWindows()

(540, 720, 3)
30


# Adaptive Threshold

In [ ]:
# 이진화 방법
threshold1 = cv2.Adaptivethreshold(src_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)